In [92]:
from tabulate import tabulate
import numpy as np


class Grid:
    def __init__(self, _type: np.dtype, def_value, M=1000, N=1000):
        self.M = M
        self.N = N
        self.arr = np.full(shape=(self.M * 2, self.N * 2), fill_value=def_value, dtype=_type)
        self.min_x, self.min_y = self.M * 2, self.N * 2
        self.max_x, self.max_y = -1, -1

    def __setitem__(self, key, value):
        x, y = key
        x += self.M
        y += self.N
        self.min_x = min(self.min_x, x)
        self.min_y = min(self.min_y, y)
        self.max_x = max(self.max_x, x)
        self.max_y = max(self.max_y, y)
        self.arr[x, y] = value

    def __getitem__(self, key):
        x, y = key
        if type(x) == int:
            x += self.M
        else:
            x = slice(x.start + self.M if x.start else None, x.stop + self.M if x.stop else None, x.step)
        if type(y) == int:
            y += self.M
        else:
            y = slice(y.start + self.N if y.start else None, y.stop + self.N if y.stop else None, y.step)
        return self.arr[x, y]

    @property
    def min(self):
        return self.min_x - self.M, self.min_y - self.N

    @property
    def max(self):
        return self.max_x + 1 - self.M, self.max_y + 1 - self.N

    @property
    def only_valuable(self):
        return self.arr[self.min_x:self.max_x + 1, self.min_y:self.max_y + 1]

    def __repr__(self):
        arr = self.only_valuable.transpose()
        return tabulate(np.c_[np.array(range(self.min_y - self.N, self.max_y + 1 - self.N)), arr],
                        headers=range(self.min_x - self.M, self.max_x + 1 - self.M))

In [90]:
def fill_up_single(grid, x, y, v):
    if grid[x, y] == b'.':
        grid[x, y] = v


def fill_up(sensor_x, sensor_y, dif_total):
    for i in range(dif_total + 1):
        for j in range(i + 1):
            fill_up_single(grid, sensor_x - j, sensor_y - dif_total + i, '#')
            fill_up_single(grid, sensor_x + j, sensor_y - dif_total + i, '#')
            fill_up_single(grid, sensor_x - j, sensor_y + dif_total - i, '#')
            fill_up_single(grid, sensor_x + j, sensor_y + dif_total - i, '#')

In [172]:
grid = Grid('|S1', '.')
with open("15_1.txt") as f:
    _input = [''.join(list(filter(lambda x: x.isdigit() or x == ' ', s))).split() for s in f.read().split("\n")]
for single_input in _input:
    sensor_x, sensor_y, beacon_x, beacon_y = map(lambda x: int(x), single_input)
    grid[sensor_x, sensor_y,] = 'S'  # S
    grid[beacon_x, beacon_y] = 'B'  # B
    total_dif = abs(sensor_x - beacon_x) + abs(sensor_y - beacon_y)

    # if sensor_y - total_dif <= 10 <= sensor_y + total_dif :
    fill_up(sensor_x, sensor_y, total_dif)
len(list(filter(lambda x: x == b'#', grid[:, 10])))

26

In [185]:
with open("15_0.txt") as f:
    _input = [''.join(list(filter(lambda x: x.isdigit() or x == ' ', s))).split() for s in f.read().split("\n")]
Y = 11 # 2000000
ranges = []
for single_input in _input:
    sensor_x, sensor_y, beacon_x, beacon_y = map(lambda x: int(x), single_input)
    total_dif = abs(sensor_x - beacon_x) + abs(sensor_y - beacon_y)
    if sensor_y - total_dif <= Y <= sensor_y + total_dif:
        dt = total_dif - abs(sensor_y - Y)
        ranges.append([sensor_x - dt, sensor_x + dt])
ranges.sort(key=lambda x: x[0])
total = 0
prev_max = (-10000000, -100000000)
for _range in ranges:
    if _range[0] >= prev_max[1]:
        total += _range[1] - _range[0]
        prev_max = _range
    if _range[0] <= prev_max[1] <= _range[1]:
        total += _range[1] - prev_max[1]
        prev_max = prev_max[1], _range[1]
    if prev_max[1] > _range[1]:
        continue
print(ranges)
print(total)

[[-3, 3], [3, 13], [11, 13], [15, 25], [15, 17]]
26


In [183]:
grid

        -8    -7    -6    -5    -4    -3    -2    -1    0    1    2    3    4    5    6    7    8    9    10    11    12    13    14    15    16    17    18    19    20    21    22    23    24    25    26    27    28
------  ----  ----  ----  ----  ----  ----  ----  ----  ---  ---  ---  ---  ---  ---  ---  ---  ---  ---  ----  ----  ----  ----  ----  ----  ----  ----  ----  ----  ----  ----  ----  ----  ----  ----  ----  ----  ----
b'-10'  .     .     .     .     .     .     .     .     .    .    #    .    .    .    .    .    .    .    .     .     .     .     .     .     .     .     .     .     .     .     .     .     .     .     .     .     .
 b'-9'  .     .     .     .     .     .     .     .     .    #    #    #    .    .    .    .    .    .    .     .     .     .     .     .     .     .     .     .     .     .     .     .     .     .     .     .     .
 b'-8'  .     .     .     .     .     .     .     .     #    #    #    #    #    .    .    .    .    .    .     .     .     .     . 

In [187]:
with open("15_0.txt") as f:
    _input = [''.join(list(filter(lambda x: x.isdigit() or x == ' ', s))).split() for s in f.read().split("\n")]
Y = 11 # 2000000
ranges = []
for y in range(0, 4000000):
    for single_input in _input:
        sensor_x, sensor_y, beacon_x, beacon_y = map(lambda x: int(x), single_input)
        total_dif = abs(sensor_x - beacon_x) + abs(sensor_y - beacon_y)
        if sensor_y - total_dif <= Y <= sensor_y + total_dif:
            dt = total_dif - abs(sensor_y - Y)
            ranges.append([sensor_x - dt, sensor_x + dt])
    ranges.sort(key=lambda x: x[0])
    for i, _range in enumerate(ranges):
        if i == 0:
            continue
        if ranges[i - 1][1] == _range[0] - 2:
            print(list(_range), y)


[15, 25] 0
[15, 25] 1
[15, 25] 2
[15, 25] 3
[15, 25] 4
[15, 25] 5
[15, 25] 6
[15, 25] 7
[15, 25] 8
[15, 25] 9
[15, 25] 10
[15, 25] 11
[15, 25] 12
[15, 25] 13
[15, 25] 14
[15, 25] 15
[15, 25] 16
[15, 25] 17
[15, 25] 18
[15, 25] 19
[15, 25] 20
[15, 25] 21
[15, 25] 22
[15, 25] 23
[15, 25] 24
[15, 25] 25
[15, 25] 26
[15, 25] 27
[15, 25] 28
[15, 25] 29
[15, 25] 30
[15, 25] 31
[15, 25] 32
[15, 25] 33
[15, 25] 34
[15, 25] 35
[15, 25] 36
[15, 25] 37
[15, 25] 38
[15, 25] 39
[15, 25] 40
[15, 25] 41
[15, 25] 42
[15, 25] 43
[15, 25] 44
[15, 25] 45
[15, 25] 46
[15, 25] 47
[15, 25] 48
[15, 25] 49
[15, 25] 50
[15, 25] 51
[15, 25] 52
[15, 25] 53
[15, 25] 54
[15, 25] 55
[15, 25] 56
[15, 25] 57
[15, 25] 58
[15, 25] 59
[15, 25] 60
[15, 25] 61
[15, 25] 62
[15, 25] 63
[15, 25] 64
[15, 25] 65
[15, 25] 66
[15, 25] 67
[15, 25] 68
[15, 25] 69
[15, 25] 70
[15, 25] 71
[15, 25] 72
[15, 25] 73
[15, 25] 74
[15, 25] 75
[15, 25] 76
[15, 25] 77
[15, 25] 78
[15, 25] 79
[15, 25] 80
[15, 25] 81
[15, 25] 82
[15, 25] 83
[1

KeyboardInterrupt: 